In [1]:
import pandas as pd
import json
from fuzzywuzzy import process,fuzz
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
from urllib.request import Request,urlopen
import requests
import urllib.parse
import time
import re
import warnings
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
url='https://www.amazon.com/ref=nav_logo'
warnings.filterwarnings("ignore")
driver = webdriver.Chrome()
driver.get(url)
def scroll(): # a function to slowly scroll down the page, this is useful when the page is not loaded properly and the wifi is slow
    total_page_height = driver.execute_script("return document.body.scrollHeight") #take height of the [age]
    browser_window_height = driver.get_window_size(windowHandle='current')['height'] #window size
    current_position = driver.execute_script('return window.pageYOffset') #current positon
    while total_page_height - current_position > browser_window_height:
        driver.execute_script(f"window.scrollTo({current_position}, {browser_window_height + current_position});")
        current_position = driver.execute_script('return window.pageYOffset')
        time.sleep(1) #this is an important step to scroll down slowly

In [31]:
import random

def generate_random_number(min_value=8, max_value=20):
    """Generate a random number between min_value and max_value, inclusive."""
    return random.randint(min_value, max_value)


generate_random_number()


In [4]:
driver.find_element("xpath",f'//*[@id="nav-hamburger-menu"]/i').click()
time.sleep(2)
driver.find_element("xpath",f'//*[@id="hmenu-content"]/ul[1]/li[11]/a[1]/div').click()
time.sleep(2.5)
driver.find_element("xpath",f'//*[@id="hmenu-content"]/ul[1]/ul[1]/li[6]/a/i').click()

In [5]:
category=[]
category_temp=driver.find_element("xpath","/html/body/div[3]/div[2]/div/ul[13]/li[3]/a")
category.append(category_temp.text)
category_temp.click()


In [6]:
subcategory=[]
try:  
    xpath = f"/html/body/div[1]/div[1]/div[1]/div[2]/div/div[3]/span/div[1]/div/div/div[2]/ul/span/span/li[6]/span/a/span"
        
        # Wait for the element to be visible and get the element
    element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, xpath)))
    subcategory.append(element.text)
    time.sleep(2.3)
    element.click()
    time.sleep(15)
    subcategory_page=driver.current_url
    req = Request(subcategory_page, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'})
    webpage = urlopen(req).read()
    time.sleep(5)
    with requests.Session() as c:
      soup = BeautifulSoup(webpage, 'html5lib')

except Exception as e:
        print(f"An error occurred with index : {e}")
      

In [26]:
product_details = {"colors": {}}

# Assuming the driver has been initialized

def get_colors():
    """
    Attempts to find color options using different methods provided.
    Returns a list of WebElement representing the color options.
    """
    try:
        # First method to get colors
        color_options = driver.find_elements(By.CSS_SELECTOR, "img.imgSwatch")
        if color_options:
            return color_options,1
        else:
            # Second method to get color if the first one fails
            color_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "a-dropdown-prompt")))
            color_options = [color_element]  # Wrap the single element in a list for consistency
            return color_options,2
    except NoSuchElementException:
        print("Color options not found.")
        return []

def get_price():
    try:

        price_element = driver.find_element(By.XPATH, '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[3]') #first way of getting price
        # Extract the price text
        price = price_element.text
        price = price.replace('\n', '.')
        print(price)
        if (price==''): #second way of getting price
            price_element = driver.find_element(By.XPATH, '//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[2]/span[2]')

    # Extract the price text

            price = price_element.text
            price = price.replace('\n', '.')
            print("Price:", price)

    except NoSuchElementException:
        try:
            print('here')
        # If the first XPath didn't find the element, try the new XPath
            price_element = driver.find_element(By.XPATH, '//*[@id="corePrice_desktop"]/div/table/tbody/tr/td[2]/span[1]/span[2]') #third way of getting price
    # Extract the price text
            price = price_element.text
  

        
        # Print the extracted price
            print("Price:", price)
            if price=='-': #fourth way of getting price
                price_element1 = driver.find_element(By.XPATH, '//*[@id="corePrice_desktop"]/div/table/tbody/tr/td[2]/span[1]/span[1]')

                price_element = driver.find_element(By.XPATH, '//*[@id="corePrice_desktop"]/div/table/tbody/tr/td[2]/span[1]/span[3]/span[2]')
    # Extract the price text
                price1=price_element1.text
                price2 = price_element.text
                price = price1 + ' - ' + price2
                print("Price:", price)
    
        except NoSuchElementException:
            price='NA'
            print("Price element not found.")


     

    return price  # Return 'NA' if all methods fail to find the price

def get_picture_urls():
    """
    Attempts to gather all picture URLs for the selected color by clicking through available buttons.
    Returns a list of picture URLs.
    """
    picture_urls = []
    index = 1
    index1=5
    while True:
        try:
            time.sleep(5)  # Sleep before attempting to find the next button to ensure the page has loaded
            #button_xpath = f'//*[@id="a-autoid-{index}"]/span/input'
            #button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
            #time.sleep(5)  # Sleep before clicking the button
            #actions = ActionChains(driver)

# Move to the element to hover over
            #actions.move_to_element(button).perform()

            #button.click()

            #time.sleep(5)  # Wait a moment for any dynamic content to load
            image_xpath = f'//*[@id="landingImage"]'
            
            image_element = driver.find_element(By.XPATH, image_xpath)
            image_url = image_element.get_attribute('src')
            picture_urls.append(image_url)
            break
            index1+=1
            index += 1

        except NoSuchElementException:
            print("No more element options found.")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break

    return picture_urls


# Main function to organize the scraping process
def scrape_product_details():
    #product_details = {"colors": {}}
    sale_price = get_price()
    time.sleep(generate_random_number())
    color_options ,num= get_colors()
    for color_option in color_options:
        # Click on the color option to load its details
        if num==1:
            time.sleep(2)
            color_option.click()
            color_text = color_option.get_attribute("alt")
        else:
            #print(color_option)
            color_text = color_option.text.strip()
            print(color_text)
        time.sleep(2)  # Wait for the page to load the color-specific details
        picture_urls = get_picture_urls()
        
        
        

        # Save the details in the product details dictionary
        product_details["colors"][color_text] = {
            "images": picture_urls,
            "price": [{"salePrice": sale_price}]
        }

    #print(product_details)
    return product_details


#scrape_product_details()

In [12]:
product_urls = []  # List to hold the extracted URLs
# Function to get all product URLs from the current page
def get_product_urls():
    divs = soup.find_all('div', class_='s-product-image-container')

    for div in divs:
    # Find the 'a' tag within the div
        a_tag = div.find('a', class_='a-link-normal')
        if a_tag:
        # Extract the href attribute
            href = a_tag['href']
        
        # Optional: Clean up the URL
        # Assuming 'base_url' is the website's base URL, e.g., "https://www.example.com"
            base_url = url
            full_url = urllib.parse.urljoin(base_url, href)
        
        # Parse the URL to remove all query parameters for cleaning
            parsed_url = urllib.parse.urlparse(full_url)
            clean_url = urllib.parse.urlunparse(parsed_url._replace(query=""))
        
        # Store the clean URL in the list
            product_urls.append(clean_url)
            
        else:
            print("No 'a' tag found within the div.")
    return product_urls

# Function to navigate to the next page
def go_to_next_page():
    # Find and click the next page button. Update the selector as needed.
    try:
        scroll()
        next_page_button = driver.find_element(By.CSS_SELECTOR, 'a.s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')

        next_page_button.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(next_page_button))
        time.sleep(2) # Wait for the page to load completely
    except NoSuchElementException:
        print("Next page button not found.")
    except TimeoutException:
        print("Page did not load in time.")


processed_count = 0
max_products = 11 #100 per 1 subcategory
while processed_count < max_products:
    #main_product_urls.append(product_urls)
    product_urls = get_product_urls()
    
    for url in product_urls:
        if processed_count >= max_products:
            break
        #driver.get(url)
        processed_count += 1
        # Here, you can add more processing for each product page
        print(f"Processed product {processed_count}: {url}")
        # Go back to the product list page if needed, or manage tabs if you open each product in a new tab
    
    if processed_count < max_products:
        print("Going to the next page.")
        go_to_next_page()

print("Finished processing 100 products.")



Processed product 1: https://www.amazon.com/sspa/click
Processed product 2: https://www.amazon.com/Amazon-Essentials-Full-Zip-Sweatshirt-X-Large/dp/B075JNL9S6/ref=sr_1_2
Processed product 3: https://www.amazon.com/Pro-Club-Heavyweight-Fleece-Hoodie/dp/B01N45SX6P/ref=sr_1_3
Processed product 4: https://www.amazon.com/NIKE-Hoodie-White-Black-Medium/dp/B00EUHNIT4/ref=sr_1_4
Processed product 5: https://www.amazon.com/Amazon-Essentials-Sherpa-Full-Zip-Sweatshirt/dp/B079PK1XT7/ref=sr_1_5
Processed product 6: https://www.amazon.com/Amazon-Essentials-Hooded-Fleece-Sweatshirt/dp/B075JNL9T3/ref=sr_1_6
Processed product 7: https://www.amazon.com/True-Religion-Buddha-Hoodie-Black/dp/B06XS74WJ2/ref=sr_1_7
Processed product 8: https://www.amazon.com/Pro-Club-Heavyweight-Pullover-Hoodie/dp/B00OK3Q08Y/ref=sr_1_8
Processed product 9: https://www.amazon.com/mens-Logo-Fleece-Hoodie-Crimson/dp/B09R3F85V6/ref=sr_1_9
Processed product 10: https://www.amazon.com/Ariat-Logo-Hoodie-Charcoal-Heather/dp/B0979DB

In [16]:

product_urls=product_urls[:11]

In [18]:
product_urls.pop(0)
product_urls

['https://www.amazon.com/Amazon-Essentials-Full-Zip-Sweatshirt-X-Large/dp/B075JNL9S6/ref=sr_1_2',
 'https://www.amazon.com/Pro-Club-Heavyweight-Fleece-Hoodie/dp/B01N45SX6P/ref=sr_1_3',
 'https://www.amazon.com/NIKE-Hoodie-White-Black-Medium/dp/B00EUHNIT4/ref=sr_1_4',
 'https://www.amazon.com/Amazon-Essentials-Sherpa-Full-Zip-Sweatshirt/dp/B079PK1XT7/ref=sr_1_5',
 'https://www.amazon.com/Amazon-Essentials-Hooded-Fleece-Sweatshirt/dp/B075JNL9T3/ref=sr_1_6',
 'https://www.amazon.com/True-Religion-Buddha-Hoodie-Black/dp/B06XS74WJ2/ref=sr_1_7',
 'https://www.amazon.com/Pro-Club-Heavyweight-Pullover-Hoodie/dp/B00OK3Q08Y/ref=sr_1_8',
 'https://www.amazon.com/mens-Logo-Fleece-Hoodie-Crimson/dp/B09R3F85V6/ref=sr_1_9',
 'https://www.amazon.com/Ariat-Logo-Hoodie-Charcoal-Heather/dp/B0979DBSGR/ref=sr_1_10',
 'https://www.amazon.com/Gap-mens-Logo-Hoodie-Sweatshirt/dp/B09R2K6FV4/ref=sr_1_11']

In [19]:
def reviews():
    def click_see_more_reviews(driver):
        try:
        # Locate the "See more reviews" button by its data-hook attribute
            see_more_reviews_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-hook="see-all-reviews-link-foot"]'))
        )
            see_more_reviews_button.click()
            time.sleep(10)  # Wait for the page to load after clicking

        # Return the current URL
            return driver.current_url
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    rev_url = click_see_more_reviews(driver)

    if rev_url:
    
        req = Request(rev_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    })
        webpage = urlopen(req).read()

        with requests.Session() as c:
            soup = BeautifulSoup(webpage, 'html5lib')
    
    
        # Store the reviews
        product_reviews = []

        while True:
    # Fetch the content from the URL
            time.sleep(8)
            if len(product_reviews) == 10:
                break
            req = Request(rev_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        })
            webpage = urlopen(req).read()

            with requests.Session() as c:
                soup = BeautifulSoup(webpage, 'html5lib')
    
            review_blocks = soup.find_all("div", class_="a-row")

            review_containers = soup.find_all("div", class_="a-section celwidget")

            for review_container in review_containers:
    # Extract the rating
                rating_tag = review_container.find("i", {"data-hook": "review-star-rating"})
                rating = float(rating_tag.find("span", class_="a-icon-alt").text.split()[0]) if rating_tag else None
    
    # Extract the review title if present
        #review_title_tag = review_container.find("a", {"data-hook": "review-title"})
        #review_title = review_title_tag.text.strip() if review_title_tag else None

    # Extract the review body text
                review_body_tag = review_container.find("span", {"data-hook": "review-body"})
    # Sometimes the actual review is nested within another span inside the review body
                review_text_tag = review_body_tag.find("span") if review_body_tag else None
                review_text = review_text_tag.text.strip() if review_text_tag else None

    # Combine review title and review text if both are present, else use what's available
        #full_review_text = review_title + " " + review_text if review_title and review_text else review_title or review_text
    
                product_reviews.append({"review": review_text or None, "rating": rating})

   
    
            try:
            # Wait for the "Next page" link to be clickable
                url = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Next page"))
    )
    
    # Click on the "Next page" link
                url.click()
                time.sleep(15)
                rev_url = driver.current_url
            except (NoSuchElementException, TimeoutException) as e:
                print("Next page link not found or not clickable.")
                break
            except Exception as e:
                print(f"An unexpected error occurred: {e}")
                break
    return product_reviews



In [22]:
def scrape_productdetails1(product_url):
    time.sleep(5)
    driver.get(product_url)
  
    time.sleep(6)
    #driver.get(product_urls)

    # Optionally, wait for some time to let the Google page load or interact with it
    #driver.implicitly_wait(5)
    product_title = driver.find_element(By.XPATH, '//*[@id="productTitle"]').text

    print(product_title)
    time.sleep(6)
    brand_text= driver.find_element(By.XPATH, '//*[@id="bylineInfo"]').text
    brand_name = brand_text.replace('Visit the ', '')
    brand_name = brand_name.replace('Brand: ', '').strip()
    print(brand_name)
    
    subcategoryy=subcategory
    Gender=[]
    if any('Men\'s' in category_word for category_word in subcategory):
        Gender.append('Male')
    
    #click_elements_sequence(driver)
    time.sleep(3)
    texts = []

# Find all span elements with the class 'a-list-item a-size-base a-color-base'
    span_elements = driver.find_elements(By.CSS_SELECTOR, "span.a-list-item.a-size-base.a-color-base")

# Loop through each span element and append its text content to the texts list
    for span in span_elements:
        texts.append(span.text.strip())

    # Join all texts into a single string
    decription_text = '.'.join(texts)

    print(decription_text)
    
    try:
    # Adjust the XPath to target the specific span element
        asin_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="detailBullets_feature_div"]/ul/li[6]/span/span[2]'))
    )
    # Extract and print the text
        asin_text = asin_element.text
        print(asin_text)
    except Exception as e:
        print(f"Error finding asin element: {e}")
    

    #productreview=reviews()

    product_details = {
        "productUrl": product_url,
        "productId": asin_text,  # Placeholder, extract the actual ID
        "gender": Gender[0],  # Determine based on the page or predefined
        "category": category,  # Determine based on the page or predefined
        "description":  decription_text,  # Extract the actual description
        "subCategory": subcategoryy,  # Determine based on the page or predefined
        "productName": product_title,
         "brandName": brand_name
       # "Product reviews": productreview
    }
 
    color_and_rating_info = scrape_product_details()
    product_details.update(color_and_rating_info)
    
    productreview=reviews()
    product_details["Product reviews"]=productreview

    return product_details


    # Assuming the scraping process is started
    #product_details = {"colors": {}}
    #product_details = scrape_product_details()
    #product details looki like this {'colors': {'Black': {'images': ['https://m.media-amazon.com/images/I/81iKJ1CM21L._AC_SL1500_.jpg',
    #'https://m.media-amazon.com/images/I/71T0oim9u7L._AC_SL1500_.jpg',
    #'https://m.media-amazon.com/images/I/71mvdpjXVvL._AC_SL1500_.jpg',
    #'https://m.media-amazon.com/images/I/81hAsGNV8+L._AC_SL1500_.jpg',
    #'https://m.media-amazon.com/images/I/611b7+trIUL._AC_SL1500_.jpg',
    #'https://m.media-amazon.com/images/I/71N3ZIN2GeL._AC_SL1500_.jpg'],
   #'price': [{'salePrice': '$14.98'}]}
    

    


    
    
  
    


In [27]:
all_product_details = {}

for index, product_url in enumerate(product_urls, start=0):
    details = scrape_productdetails1(product_url)
    all_product_details[f"Product {index}"] = {"product details": details}

Amazon Essentials Men's Full-Zip Hooded Fleece Sweatshirt (Available in Big & Tall)
Amazon Essentials Store
REGULAR FIT: Comfortable, easy fit through the shoulders, chest, and waist.BRUSHED FLEECE: 8.3 oz soft brushed-back cotton-polyester blend fleece for staying cozy and keeping warm..HOODED SWEATSHIRT: A classic everyday essential, this hooded sweatshirt is casual and comfortable. Throw it on for running errands or layer it with a collared shirt for a more dressed up look..DETAILS: Features three panel lined hood with metal grommets and adjustable drawcord, full length center front zipper, and hand pockets. Athletic rib trim at sleeve and bottom hem.
B07Q1LC2K2
here
Price: -
Price: $16.40 - $29.40
No more element options found.
No more element options found.
No more element options found.
No more element options found.
No more element options found.
No more element options found.
No more element options found.
No more element options found.
No more element options found.
No more el

In [29]:
all_product_details

{'Product 0': {'product details': {'productUrl': 'https://www.amazon.com/Amazon-Essentials-Full-Zip-Sweatshirt-X-Large/dp/B075JNL9S6/ref=sr_1_2',
   'productId': 'B07Q1LC2K2',
   'gender': 'Male',
   'category': ['Clothing'],
   'description': 'REGULAR FIT: Comfortable, easy fit through the shoulders, chest, and waist.BRUSHED FLEECE: 8.3 oz soft brushed-back cotton-polyester blend fleece for staying cozy and keeping warm..HOODED SWEATSHIRT: A classic everyday essential, this hooded sweatshirt is casual and comfortable. Throw it on for running errands or layer it with a collared shirt for a more dressed up look..DETAILS: Features three panel lined hood with metal grommets and adjustable drawcord, full length center front zipper, and hand pockets. Athletic rib trim at sleeve and bottom hem.',
   'subCategory': ["Men's Fashion Hoodies & Sweatshirts"],
   'productName': "Amazon Essentials Men's Full-Zip Hooded Fleece Sweatshirt (Available in Big & Tall)",
   'brandName': 'Amazon Essentials

In [30]:
with open('product_details.json', 'w') as outfile:
    json.dump(all_product_details, outfile, indent=4)